## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [47]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [48]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [49]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210209.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [50]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,400230,16600,34500,451330,390077,0.864283,151577,2021-02-08
1,Aragón,68175,2900,5400,76475,75586,0.988375,25222,2021-02-08
2,Asturias,69935,2900,4200,77035,72787,0.944856,31944,2021-02-08
3,Baleares,35460,1500,4800,41760,36655,0.877754,14072,2021-02-08
4,Canarias,79530,3200,8900,91630,85250,0.930372,29047,2021-02-08
5,Cantabria,33495,1300,2400,37195,35347,0.950316,13470,2021-02-08
6,Castilla y Leon,149475,6200,9800,165475,161781,0.977676,64568,2021-02-08
7,Castilla La Mancha,101655,4200,8300,114155,106646,0.934221,33207,2021-02-08
8,Cataluña,345770,14300,31800,391870,328789,0.839026,129651,2021-02-08
9,C. Valenciana,175690,7000,20600,203290,196489,0.966545,87353,2021-02-08


..hacemos una copia para hacerle las transformaciones necesarias:

In [60]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [61]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [62]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [63]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [64]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [67]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,400230,16600,34500,451330,390077,0.864283,151577,8/2/21,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,68175,2900,5400,76475,75586,0.988375,25222,8/2/21,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,69935,2900,4200,77035,72787,0.944856,31944,8/2/21,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,35460,1500,4800,41760,36655,0.877754,14072,8/2/21,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,79530,3200,8900,91630,85250,0.930372,29047,8/2/21,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,33495,1300,2400,37195,35347,0.950316,13470,8/2/21,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,149475,6200,9800,165475,161781,0.977676,64568,8/2/21,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,101655,4200,8300,114155,106646,0.934221,33207,8/2/21,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,345770,14300,31800,391870,328789,0.839026,129651,8/2/21,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,175690,7000,20600,203290,196489,0.966545,87353,8/2/21,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [68]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [69]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [70]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [71]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,9/2/21,Andalucía,400230,16600,34500,451330,390077,0.864283,151577,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,9/2/21,Aragón,68175,2900,5400,76475,75586,0.988375,25222,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,9/2/21,Asturias,69935,2900,4200,77035,72787,0.944856,31944,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,9/2/21,Baleares,35460,1500,4800,41760,36655,0.877754,14072,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,9/2/21,Canarias,79530,3200,8900,91630,85250,0.930372,29047,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,9/2/21,Cantabria,33495,1300,2400,37195,35347,0.950316,13470,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,9/2/21,Castilla y Leon,149475,6200,9800,165475,161781,0.977676,64568,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,9/2/21,Castilla La Mancha,101655,4200,8300,114155,106646,0.934221,33207,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,9/2/21,Cataluña,345770,14300,31800,391870,328789,0.839026,129651,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,9/2/21,C. Valenciana,175690,7000,20600,203290,196489,0.966545,87353,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [72]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [73]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [77]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source,Dosis entregadas AstraZeneca
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...,NaN
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...,NaN
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...,NaN
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...,NaN
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,9/2/21,Navarra,33495.0,1300.0,37495,35585,0.949060,12509.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...,2700.0
516,9/2/21,País Vasco,91080.0,3600.0,103780,75634,0.728792,26299.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...,9100.0
517,9/2/21,Ceuta,1980.0,200.0,2580,2218,0.859690,964.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...,400.0
518,9/2/21,Melilla,1980.0,200.0,2580,2399,0.929845,1080.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...,400.0


concatenamos...

In [75]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [78]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas', 'Dosis administradas',
       '% sobre entregadas', 'Total pauta completada',
       'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source',
       'Dosis entregadas AstraZeneca'],
      dtype='object')

In [79]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [80]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,9/2/21,Navarra,33495.0,1300.0,2700.0,37495,35585,0.949060,12509.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
516,9/2/21,País Vasco,91080.0,3600.0,9100.0,103780,75634,0.728792,26299.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
517,9/2/21,Ceuta,1980.0,200.0,400.0,2580,2218,0.859690,964.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
518,9/2/21,Melilla,1980.0,200.0,400.0,2580,2399,0.929845,1080.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [81]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [82]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)